In [17]:
import numpy as np
import pandas as pd
import cPickle as pickle
from image_processing_new import get_paths, get_domi_color, image_featurizer
from collections import defaultdict
from scipy.spatial.distance import cosine, euclidean
from sklearn.feature_extraction.text import TfidfVectorizer

In [3]:
import os.path
import matplotlib.pyplot as plt
import time
% matplotlib inline

# Coffee table

In [6]:
category = 'coffee_table'
paths = get_paths(category)
len(paths)

start_time =  time.time()
# Get dominant color:
domi_color_dict = get_domi_color(paths, category)
time_1 =  time.time()
print 'get_domi_color: ', time_1 - start_time

# of grayscaled photos:  0
get_domi_color:  1576.44073892


//anaconda/lib/python2.7/site-packages/skimage/util/dtype.py:111: UserWarning: Possible precision loss when converting from float64 to uint8
  "%s to %s" % (dtypeobj_in, dtypeobj))


In [8]:
# Get PCAed features and pickle transformers:
feature_dict_pca, pca_scaler, pca_model = image_featurizer(category, pca=True)

path = 'wayfair/pickle/' + category + '_pca_scaler.pkl'
with open(path, 'w') as f:
    pickle.dump(pca_scaler, f)
path = 'wayfair/pickle/' + category + '_pca_model.pkl'
with open(path, 'w') as f:
    pickle.dump(pca_model, f)

In [12]:
# Building dataframe from dictionaries:
domi_pca_dict = defaultdict(dict)
for i in domi_color_dict:
    domi_pca_dict[i]['domi'] = domi_color_dict[i]
for i in feature_dict_pca:
    domi_pca_dict[i]['pca'] = feature_dict_pca[i]

domi_pca_df = pd.DataFrame(domi_pca_dict).T
domi_pca_df = domi_pca_df.reset_index()
domi_pca_df.rename(columns={'index': 'path'}, inplace=True)
domi_pca_df['product_id'] = domi_pca_df['path'].apply(lambda x:x.split('.')[0].split('_')[-2])

path = 'wayfair/' + category +'.json'
products_df = pd.read_json(path)

products_df_small = products_df[['product_id', 'title', 'price', 'url', 'description_all', 'rating_avg', 'rating_count']]

all_info_df = pd.merge(domi_pca_df, products_df_small, how='inner', left_on='product_id', 
                            right_on='product_id')

In [19]:
# Train a tfidf vectorizer & pickle it:
X = all_info_df['description_all']
tfidf = TfidfVectorizer(strip_accents='unicode', stop_words='english', max_df=0.8, 
                        max_features=1000, ngram_range = (1,2))
tfidf_matrix = tfidf.fit_transform(X).todense()

path = 'wayfair/pickle/' + category + '_tfidf.pkl'
with open(path, 'w') as f:
    pickle.dump(tfidf, f)

In [20]:
# Add tfide vectors to the dataframe:
tfidf_matrix = np.array(tfidf_matrix)

tfidf_dict = defaultdict(dict)
for i in xrange(tfidf_matrix.shape[0]):
    index = X.index[i]
    tfidf_dict[index]['tfidf_vec'] = tfidf_matrix[i,:]
    
tfidf_df = pd.DataFrame(tfidf_dict).T
tfidf_df = tfidf_df.reset_index()
tfidf_df.rename(columns={'index': 'ixx'}, inplace=True)

all_info_df = pd.merge(all_info_df, tfidf_df, how='inner', left_index=True, right_on='ixx')
all_info_df.drop('ixx', axis=1, inplace=True)

In [22]:
# some final clearning & save to json:
all_info_df = all_info_df[all_info_df['domi'].notnull()]
all_info_df = all_info_df[all_info_df['pca'].notnull()]
all_info_df = all_info_df[all_info_df['description_all'].notnull()]

path = 'wayfair/' + category + '_vec_info.json'
all_info_df.to_json(path)

In [28]:
all_info_df.head(2)

,path,domi,pca,product_id,title,price,url,description_all,rating_avg,rating_count,tfidf_vec
0,coffee_table_AAI1711_0.jpg,"[0.53298166968, 0.350660610644, 0.208815895222]","[-29.7162485019, -51.6280376336, 12.5068592933...",AAI1711,Coffee Table,539.99,http://www.wayfair.com/AA-Importing-Coffee-Tab...,\n\nFeatures\n\nOne drawer\nMedium stained fin...,4.8,5,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
1,coffee_table_AALA1038_0.jpg,"[0.254172877804, 0.209493494858, 0.168938476459]","[-5.55174227093, -53.3880001647, 47.4155327017...",AALA1038,Downtown Square Cocktail Table,689.99,http://www.wayfair.com/Caravel-Downtown-Square...,"Proudly made in Wisconsin, Downtown provides a...",NaN,NaN,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."


# Write the whole process into a .py file and process images from all other categories!

Done:
* sofa
* coffee_table
* dining
* office
* bookcase
* bed
* nightstand
* dresser

In [25]:
from all_vec_info_database_builder import build_all_vec_info_json

In [26]:
category = 'dining'
build_all_vec_info_json(category)

11018
# of grayscaled photos:  0
get_domi_color:  4529.52110481


//anaconda/lib/python2.7/site-packages/skimage/io/_io.py:159: UserWarning: wayfair/images/dining/white/dining_chair_KTL1543_13.jpg is a low contrast image
  warnings.warn('%s is a low contrast image' % fname)


### To do: delete "dining_chair_KTL1543_13.jpg"?

In [29]:
test_df = pd.read_json('wayfair/dining_vec_info.json')
test_df.info()

<class 'pandas.core.frame.DataFrame'>
Float64Index: 6825 entries, 0.0 to 999.0
Data columns (total 11 columns):
description_all    6825 non-null object
domi               6825 non-null object
path               6825 non-null object
pca                6825 non-null object
price              6825 non-null float64
product_id         6825 non-null object
rating_avg         4305 non-null float64
rating_count       4305 non-null float64
tfidf_vec          6825 non-null object
title              6825 non-null object
url                6825 non-null object
dtypes: float64(3), object(8)
memory usage: 639.8+ KB


In [33]:
test_df.tail(2)

,description_all,domi,path,pca,price,product_id,rating_avg,rating_count,tfidf_vec,title,url
998,"The Eurostyle Max Leather Chair (Set of 2), wi...","[0.3652446444, 0.3651046761, 0.3724030829]",dining_chair_EY1790_13.jpg,"[-35.892256317, -0.6223099796, 2.5057500055, -...",558,EY1790,4.6,5,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",Max Leather Chair,http://www.wayfair.com/Eurostyle-Max-Leather-C...
999,"The Eurostyle Max Leather Chair (Set of 2), wi...","[0.6018939243, 0.5269973979, 0.4495489969]",dining_chair_EY1790_14.jpg,"[-29.6310459973, -9.1264292381, 6.6028668314, ...",558,EY1790,4.6,5,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",Max Leather Chair,http://www.wayfair.com/Eurostyle-Max-Leather-C...


In [34]:
category = 'office'
build_all_vec_info_json(category)

11511
# of grayscaled photos:  0
Time to run get_domi_color (s):  4921.02584505


In [35]:
category = 'bookcase'
build_all_vec_info_json(category)

4332
# of grayscaled photos:  0
Time to run get_domi_color (s):  1705.985111


In [36]:
category = 'bed'
build_all_vec_info_json(category)

4747
# of grayscaled photos:  1
Time to run get_domi_color (s):  1905.26973486


In [37]:
category = 'nightstand'
build_all_vec_info_json(category)

3998
# of grayscaled photos:  0
Time to run get_domi_color (s):  1590.58482313


In [38]:
category = 'dresser'
build_all_vec_info_json(category)

3712
# of grayscaled photos:  0
Time to run get_domi_color (s):  1449.16593194
